In [1]:
import csv
import requests
import sys
import os
import re
import time
import itertools
import json
from pandas import DataFrame
import pandas as pd
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper

In [2]:
with open('mapRef.tsv', 'r') as mart, open('temp_filtered_mapRef.tsv', 'w', newline='') as filtered:

    # define reader and writer objects
    reader = csv.reader(mart,delimiter="\t", skipinitialspace=True)
    writer = csv.writer(filtered, delimiter="\t")

    # iterate and write rows based on condition
    for i in reader:
        if i[0] != 'MT':
            writer.writerow(i)

In [3]:
with open('temp_filtered_mapRef.tsv', 'r') as mart, open('filtered_mapRef.tsv', 'w', newline='') as filtered:

    # define reader and writer objects
    reader = csv.reader(mart,delimiter="\t", skipinitialspace=True)
    writer = csv.writer(filtered, delimiter="\t")

    # iterate and write rows based on condition
    for i in reader:
        if i[0] != 'Y':
            writer.writerow(i)

In [4]:
tsv_file = open("filtered_mapRef.tsv")
mart_tsv = csv.reader(tsv_file, delimiter="\t", quotechar='"')
df = DataFrame(mart_tsv)

In [7]:
df.shape

(93628, 7)

In [5]:
dfRef = df[df[6].astype(bool)]

In [8]:
dfRef.shape

(15543, 7)

In [6]:
dfRef.head()

,0,1,2,3,4,5,6
0,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Gene stable ID,Transcript stable ID,Protein stable ID,RefSeq match transcript
18,1,944203,959309,ENSG00000188976,ENST00000327044,ENSP00000317992,NM_015658.4
19,1,960584,965719,ENSG00000187961,ENST00000338591,ENSP00000343930,NM_198317.3
21,1,966482,975865,ENSG00000187583,ENST00000379410,ENSP00000368720,NM_032129.3
28,1,998962,1000172,ENSG00000188290,ENST00000304952,ENSP00000304595,NM_021170.4


In [9]:
dfClinvar = pd.read_csv('result1.csv', index_col=0)

In [10]:
dfClinvar = DataFrame(dfClinvar)

In [11]:
dfClinvar.shape

(31008, 10)

In [12]:
dfClinvar.head()

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,MC,CLNHGVS,CLNSIG
79,1,943995,950448,C,T,NaN,NaN,SO:0001587|nonsense,NC_000001.11:g.943995C>T,Pathogenic
82,1,944051,961217,T,G,NaN,NaN,SO:0001587|nonsense,NC_000001.11:g.944051T>G,Uncertain_significance
125,1,1014143,183381,C,T,NaN,NaN,SO:0001587|nonsense,NC_000001.11:g.1014143C>T,Pathogenic
141,1,1014359,161454,G,T,NaN,NaN,SO:0001587|nonsense,NC_000001.11:g.1014359G>T,Pathogenic
248,1,1041582,126556,C,T,NaN,NaN,SO:0001587|nonsense,NC_000001.11:g.1041582C>T,Pathogenic


In [13]:
hgvs_g = 'NC_000001.11:g.1014359G>T'
hp = hgvs.parser.Parser()
var_g = hp.parse_hgvs_variant(hgvs_g)
var_g

SequenceVariant(ac=NC_000001.11, type=g, posedit=1014359G>T, gene=None)

In [14]:
hdp = hgvs.dataproviders.uta.connect()
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name = 'GRCh38', alt_aln_method = "splign", replace_reference = True)

In [15]:
transcripts = am.relevant_transcripts(var_g)
sorted(transcripts)

['NM_005101.3']

In [16]:
var_c = am.g_to_c(var_g, transcripts[0])
str(var_c)

'NM_005101.3:c.379G>T'

In [17]:
var_p = am.c_to_p(var_c)
str(var_p)

'NP_005092.1:p.(Glu127Ter)'

In [52]:
column = ['CHROM', 'POS', 'REF', 'ALT', 'MC', 'CLNSIG', 'NC', 'NM','NP','ENST', 'ProteinSeq', 'Distance', 'Percentage Remaining']
dfMap = DataFrame(columns = column)

In [53]:
import itertools
limit = 5

In [54]:
data = []
for index, row in itertools.islice(dfClinvar.iterrows(), limit):
    values = []
    values.append(row[0])
    values.append(row[1])
    values.append(row[3])
    values.append(row[4])
    values.append(row[7])
    values.append(row[9])
    values.append(row[8])
    hgvs_g = str(row[8])
    hp = hgvs.parser.Parser()
    var_g = hp.parse_hgvs_variant(hgvs_g)
    transcripts = am.relevant_transcripts(var_g)
    var_c = am.g_to_c(var_g, transcripts[0])
    values.append(str(var_c))
    var_p = am.c_to_p(var_c)
    values.append(str(var_p))
    time.sleep(3)
    
    zipped = zip(column, values)
    dictionary = dict(zipped)
    data.append(dictionary)

In [55]:
dfMap = dfMap.append(data,True)
dfMap['ENST'] = dfMap['ENST'].fillna('0')

In [56]:
dfMap.head()
dfMap.shape

(5, 13)

In [57]:
for index, row in dfMap.iterrows():
    NM = row[7].split('.')[0]
    Match = ""
    for x, y in dfRef.iterrows():
        Match = y[6].split('.')[0]
        if NM == Match:
            print("its a match")
            print('Transcript: ' + NM)
            print('Transcript2: ' + Match)
            print(row[9])
            print(y[4])
            dfMap.loc[index, 'ENST'] = y[4]

its a match
Transcript: NM_005101
Transcript2: NM_005101
0
ENST00000649529
its a match
Transcript: NM_005101
Transcript2: NM_005101
0
ENST00000649529
its a match
Transcript: NM_198576
Transcript2: NM_198576
0
ENST00000379370


In [58]:
#Gets a protein sequence in FASTA format given the ensembl transcript ID
def get_protein_sequence(canonical_transcript):
    server = "http://grch37.rest.ensembl.org"
    ext = "/sequence/id/" + canonical_transcript + "?content-type=text/x-fasta;type=protein"
    r = requests.get(server + ext, headers={"Content-Type": "text/x-fasta"})
    time.sleep(3)
    
    if not r.ok:
        try:
            r.raise_for_status()
            return "error"
        except requests.exceptions.HTTPError: #I should catch the error too or print, to find out which specific genes
            pass
    seqlist = r.text.split("\n", 1)
    
    if len(seqlist) == 2: #A lot of these transcripts are noncoding or introns, don't know why
        sequence = seqlist[1]
        sequence = sequence.replace("\n", "")
        return sequence
    #print("Sequence found")

#Adapted from https://github.com/xjenny2/phospho-programs/blob/master/ensembl.py

In [59]:
for index, row in dfMap.iterrows():
    if row[9] != 0:
        dfMap.loc[index, 'ProteinSeq'] = get_protein_sequence(row[9])

In [60]:
import re

In [62]:
for index, row in dfMap.iterrows():
    Protein_Mutation_Position = int(re.findall(r'\d+', row[8])[-1])
    if row[10] != None:
        print(row[10][Protein_Mutation_Position - 1]) #Index and position are difference; Index = Position - 1
        s = row[10][Protein_Mutation_Position - 1:-1]
        print(s)
        distance = 0
        retained = 0
        for z, item in enumerate(s):
            if item == 'M':
                distance = z
                ran = s[z:-1]
                retained = len(ran)/len(row[10])
                break
        dfMap.loc[index, 'Distance'] = distance
        dfMap.loc[index, 'Percentage Remaining'] = retained

Q
QAPVCGDDGVTYENDCVMGRSGAARGLLLQKVRSGQCQGRDQCPEPCRFNAVCLSRRGRPRCSCDRVTCDGAYRPVCAQDGRTYDSDCWRQQAECRQQRAIPSKHQGPCDQAPSPCLGVQCAFGATCAVKNGQAACECLQACSSLYDPVCGSDGVTYGSACELEATACTLGREIQVARKGPCDRCGQCRFGALCEAETGRCVCPSECVALAQPVCGSDGHTYPSECMLHVHACTHQISLHVASAGPCETCGDAVCAFGAVCSAGQCVCPRCEHPPPGPVCGSDGVTYGSACELREAACLQQTQIEEARAGPCEQAECGSGGSGSGEDGDCEQELCRQRGGIWDEDSEDGPCVCDFSCQSVPGSPVCGSDGVTYSTECELKKARCESQRGLYVAAQGACRGPTFAPLPPVAPLHCAQTPYGCCQDNITAARGVGLAGCPSACQCNPHGSYGGTCDPATGQCSCRPGVGGLRCDRCEPGFWNFRGIVTDGRSGCTPCSCDPQGAVRDDCEQMTGLCSCKPGVAGPKCGQCPDGRALGPAGCEADASAPATCAEMRCEFGARCVEESGSAHCVCPMLTCPEANATKVCGSDGVTYGNECQLKTIACRQGLQISIQSLGPCQEAVAPSTHPTSASVTVTTPGLLLSQALPAPPGALPLAPSSTAHSQTTPPPSSRPRTTASVPRTTVWPVLTVPPTAPSPAPSLVASAFGESGSTDGSSDEELSGDQEASGGGSGGLEPLEGSSVATPGPPVERASCYNSALGCCSDGKTPSLDAEGSNCPATKVFQGVLELEGVEGQELFYTPEMADPKSELFGETARSIESTLDDLFRNSDVKKDFRSVRLRDLGPGKSVRAIVDVHFDPTTAFRAPDVARALLRQIQVSRRRSLGVRRPLQEHVRFMDFDWFPAFITGATSGAIAAGATARATTASRLPSSAVTPRAPHPSHTSQPVAKTTAAPTTRRPPTTAPSRVPGRRPPAPQQPPKPCDSQPCFHGGTCQDWALG

In [63]:
dfMap.head()

,CHROM,POS,REF,ALT,MC,CLNSIG,NC,NM,NP,ENST,ProteinSeq,Distance,Percentage Remaining
0,1,943995,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.943995C>T,NM_152486.2:c.1888C>T,NP_689699.2:p.(Arg630Ter),0,None,NaN,NaN
1,1,944051,T,G,SO:0001587|nonsense,Uncertain_significance,NC_000001.11:g.944051T>G,NM_152486.2:c.1944T>G,NP_689699.2:p.(Tyr648Ter),0,None,NaN,NaN
2,1,1014143,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,NP_005092.1:p.(Gln55Ter),ENST00000649529,None,NaN,NaN
3,1,1014359,G,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,NP_005092.1:p.(Glu127Ter),ENST00000649529,None,NaN,NaN
4,1,1041582,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1041582C>T,NM_198576.3:c.1057C>T,NP_940978.2:p.(Gln353Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,17,0.818582


In [65]:
dfMap.to_csv('results.csv')